In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals


import numpy as np
import pandas as pd
%load_ext autoreload
%autoreload 2
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm

from helpers import count_unique_words, count_unique_ngrams, \
            build_unique_ngrams, create_sentence_vectors, create_sentence_vectors_submission

import sys

import tensorflow as tf
from tensorflow import keras

import gensim   # Not sure whether it is better to use gensim or tensorflow :/
import logging
from gensim.models.phrases import Phrases, Phraser

import multiprocessing

from gensim.models import Word2Vec

from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Embedding, Input, Conv1D, Dense, MaxPooling1D, Flatten
from tensorflow.keras.models import Model
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
 

sys.path.append('../')

/home/stefano/anaconda3/envs/tf-gpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/stefano/anaconda3/envs/tf-gpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/stefano/anaconda3/envs/tf-gpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/stefano/anaconda3/envs/tf-gpu/lib/pyt

In [2]:
df = pd.read_pickle("dataframes/full_df_cleaned_train_0_8.pickle")

In [3]:
df_test = pd.read_pickle("dataframes/full_df_cleaned_test_0_2.pickle")

In [4]:
count_unique_words(df)

452521

In [5]:
w2v_model = Word2Vec.load("models/w2v_model_epochs_5_win_5_cbow_250.model")

2019-12-18 15:20:57,602 : INFO : loading Word2Vec object from models/w2v_model_epochs_5_win_5_cbow_250.model
2019-12-18 15:20:58,275 : INFO : loading wv recursively from models/w2v_model_epochs_5_win_5_cbow_250.model.wv.* with mmap=None
2019-12-18 15:20:58,276 : INFO : loading vectors from models/w2v_model_epochs_5_win_5_cbow_250.model.wv.vectors.npy with mmap=None
2019-12-18 15:20:59,135 : INFO : setting ignored attribute vectors_norm to None
2019-12-18 15:20:59,136 : INFO : loading vocabulary recursively from models/w2v_model_epochs_5_win_5_cbow_250.model.vocabulary.* with mmap=None
2019-12-18 15:20:59,137 : INFO : loading trainables recursively from models/w2v_model_epochs_5_win_5_cbow_250.model.trainables.* with mmap=None
2019-12-18 15:20:59,138 : INFO : loading syn1neg from models/w2v_model_epochs_5_win_5_cbow_250.model.trainables.syn1neg.npy with mmap=None
2019-12-18 15:20:59,978 : INFO : setting ignored attribute cum_table to None
2019-12-18 15:20:59,978 : INFO : loaded models/w

In [6]:
w2v_model.wv.word_vec("love").shape

(250,)

In [7]:
import numpy as np

def create_embedding_matrix(w2v_model, word_index):
    vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index

    ## We can assume love is always present in our vocabulary ahaha
    embedding_matrix = np.zeros((vocab_size, w2v_model.wv.word_vec("love").shape[0]))  
    
    for word in w2v_model.wv.vocab:
        vector = w2v_model.wv.word_vec(word)
        if word in word_index:
            idx = word_index[word] 
            embedding_matrix[idx] = np.array(
                vector, dtype=np.float32)

    return embedding_matrix

In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [9]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df.sentence)

In [10]:
X_train = tokenizer.texts_to_sequences(df.sentence)

In [11]:
X_test = tokenizer.texts_to_sequences(df_test.sentence)

In [12]:
vocab_size = len(tokenizer.word_index) + 1
vocab_size

396986

In [16]:
X_train[X_train[0] == '#']

[15, 1810, 8634, 2884, 3383, 7]

In [17]:
df.iloc[8634].sentence

"want pottermore let ! can't obsessed let #nerdproblems"

In [13]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [14]:
def max_len(X):
    maxlen = 0
    for el in X:
        maxlen = maxlen if len(el) < maxlen else len(el)
    return maxlen
maxlen = int(max_len(X_train)/2)
print(maxlen)

22


In [15]:
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)

In [16]:
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [17]:
print(X_train[0, :])

[  15 1810 8634 2884 3383    7    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0]


In [18]:
embedding_dim = 250

In [19]:
embedding_matrix = create_embedding_matrix(
    w2v_model,
    tokenizer.word_index)

In [20]:
nonzero_elements = np.count_nonzero(np.count_nonzero(embedding_matrix, axis=1))
print(nonzero_elements / vocab_size)
len(embedding_matrix)

0.7895593295481452


396986

In [21]:
# Convert -1 in 0 (otherwise it doesn't work)
y_train = np.where(df.label == 1, 1, 0)
y_test = np.where(df_test.label == 1, 1, 0)

In [23]:
for i in range(50, 251, 50):
    model = Sequential()
    model.add(layers.Embedding(vocab_size, embedding_dim,
                               input_length=maxlen,
                               weights=[embedding_matrix],
                               trainable=False))
    model.add(layers.Conv1D(i, 3, activation='relu'))   ## Maybe I should increase the kernel window (currently only 5)
    model.add(layers.GlobalMaxPooling1D())
    model.add(layers.Dense(60, activation='relu'))
    model.add(layers.Dense(60, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    model.summary()
    filepath="models/convolutional_nn_layers_of_size_{}_{}".format(i, 60)
    checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    callbacks_list = [checkpoint]

    history = model.fit(X_train, y_train,
                        epochs=40,
                        verbose=True,
                        callbacks=callbacks_list,
                        validation_data=(X_test, y_test),
                        batch_size=128)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 22, 250)           99246500  
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 20, 50)            37550     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 50)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 60)                3060      
_________________________________________________________________
dense_4 (Dense)              (None, 60)                3660      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 61        
Total params: 99,290,831
Trainable params: 44,331
Non-trainable params: 99,246,500
___________________________________________________________

Epoch 24/40
1999360/2000000 [============================>.] - ETA: 0s - loss: 0.3644 - acc: 0.8305
Epoch 00024: val_acc did not improve from 0.81877
2000000/2000000 [==============================] - 40s 20us/sample - loss: 0.3644 - acc: 0.8305 - val_loss: 0.3879 - val_acc: 0.8187
Epoch 25/40
1997184/2000000 [============================>.] - ETA: 0s - loss: 0.3640 - acc: 0.8306
Epoch 00025: val_acc did not improve from 0.81877
2000000/2000000 [==============================] - 40s 20us/sample - loss: 0.3640 - acc: 0.8306 - val_loss: 0.3901 - val_acc: 0.8184
Epoch 26/40
1997312/2000000 [============================>.] - ETA: 0s - loss: 0.3637 - acc: 0.8308
Epoch 00026: val_acc did not improve from 0.81877
2000000/2000000 [==============================] - 40s 20us/sample - loss: 0.3637 - acc: 0.8308 - val_loss: 0.3869 - val_acc: 0.8184
Epoch 27/40
1997568/2000000 [============================>.] - ETA: 0s - loss: 0.3633 - acc: 0.8311
Epoch 00027: val_acc did not improve from 0.81877
2

Epoch 8/40
1998464/2000000 [============================>.] - ETA: 0s - loss: 0.3617 - acc: 0.8322
Epoch 00008: val_acc improved from 0.82026 to 0.82079, saving model to models/convolutional_nn_layers_of_size_100_60
2000000/2000000 [==============================] - 42s 21us/sample - loss: 0.3617 - acc: 0.8322 - val_loss: 0.3820 - val_acc: 0.8208
Epoch 9/40
1998720/2000000 [============================>.] - ETA: 0s - loss: 0.3597 - acc: 0.8333
Epoch 00009: val_acc improved from 0.82079 to 0.82114, saving model to models/convolutional_nn_layers_of_size_100_60
2000000/2000000 [==============================] - 42s 21us/sample - loss: 0.3597 - acc: 0.8333 - val_loss: 0.3818 - val_acc: 0.8211
Epoch 10/40
1997568/2000000 [============================>.] - ETA: 0s - loss: 0.3578 - acc: 0.8345
Epoch 00010: val_acc did not improve from 0.82114
2000000/2000000 [==============================] - 41s 21us/sample - loss: 0.3579 - acc: 0.8345 - val_loss: 0.3818 - val_acc: 0.8211
Epoch 11/40
1997824

Epoch 36/40
1999488/2000000 [============================>.] - ETA: 0s - loss: 0.3385 - acc: 0.8457
Epoch 00036: val_acc did not improve from 0.82212
2000000/2000000 [==============================] - 41s 21us/sample - loss: 0.3385 - acc: 0.8457 - val_loss: 0.3960 - val_acc: 0.8171
Epoch 37/40
1997952/2000000 [============================>.] - ETA: 0s - loss: 0.3382 - acc: 0.8459
Epoch 00037: val_acc did not improve from 0.82212
2000000/2000000 [==============================] - 41s 21us/sample - loss: 0.3382 - acc: 0.8458 - val_loss: 0.3915 - val_acc: 0.8203
Epoch 38/40
1998848/2000000 [============================>.] - ETA: 0s - loss: 0.3376 - acc: 0.8463
Epoch 00038: val_acc did not improve from 0.82212
2000000/2000000 [==============================] - 41s 21us/sample - loss: 0.3376 - acc: 0.8463 - val_loss: 0.3944 - val_acc: 0.8199
Epoch 39/40
1999360/2000000 [============================>.] - ETA: 0s - loss: 0.3374 - acc: 0.8463
Epoch 00039: val_acc did not improve from 0.82212
2

Epoch 19/40
1997696/2000000 [============================>.] - ETA: 0s - loss: 0.3348 - acc: 0.8465
Epoch 00019: val_acc did not improve from 0.82397
2000000/2000000 [==============================] - 43s 22us/sample - loss: 0.3348 - acc: 0.8464 - val_loss: 0.3879 - val_acc: 0.8228
Epoch 20/40
1997056/2000000 [============================>.] - ETA: 0s - loss: 0.3337 - acc: 0.8471
Epoch 00020: val_acc did not improve from 0.82397
2000000/2000000 [==============================] - 43s 21us/sample - loss: 0.3337 - acc: 0.8471 - val_loss: 0.3879 - val_acc: 0.8220
Epoch 21/40
1998848/2000000 [============================>.] - ETA: 0s - loss: 0.3326 - acc: 0.8476
Epoch 00021: val_acc did not improve from 0.82397
2000000/2000000 [==============================] - 43s 22us/sample - loss: 0.3326 - acc: 0.8476 - val_loss: 0.3955 - val_acc: 0.8210
Epoch 22/40
1999744/2000000 [============================>.] - ETA: 0s - loss: 0.3317 - acc: 0.8482
Epoch 00022: val_acc did not improve from 0.82397
2

1997824/2000000 [============================>.] - ETA: 0s - loss: 0.3722 - acc: 0.8263
Epoch 00003: val_acc improved from 0.81852 to 0.82009, saving model to models/convolutional_nn_layers_of_size_200_60
2000000/2000000 [==============================] - 44s 22us/sample - loss: 0.3722 - acc: 0.8263 - val_loss: 0.3825 - val_acc: 0.8201
Epoch 4/40
1998080/2000000 [============================>.] - ETA: 0s - loss: 0.3647 - acc: 0.8305
Epoch 00004: val_acc improved from 0.82009 to 0.82228, saving model to models/convolutional_nn_layers_of_size_200_60
2000000/2000000 [==============================] - 44s 22us/sample - loss: 0.3647 - acc: 0.8305 - val_loss: 0.3785 - val_acc: 0.8223
Epoch 5/40
1999488/2000000 [============================>.] - ETA: 0s - loss: 0.3588 - acc: 0.8336
Epoch 00005: val_acc improved from 0.82228 to 0.82343, saving model to models/convolutional_nn_layers_of_size_200_60
2000000/2000000 [==============================] - 44s 22us/sample - loss: 0.3588 - acc: 0.8336 -

1997568/2000000 [============================>.] - ETA: 0s - loss: 0.3105 - acc: 0.8607
Epoch 00031: val_acc did not improve from 0.82386
2000000/2000000 [==============================] - 43s 22us/sample - loss: 0.3105 - acc: 0.8607 - val_loss: 0.4153 - val_acc: 0.8192
Epoch 32/40
1997440/2000000 [============================>.] - ETA: 0s - loss: 0.3097 - acc: 0.8613
Epoch 00032: val_acc did not improve from 0.82386
2000000/2000000 [==============================] - 43s 21us/sample - loss: 0.3097 - acc: 0.8613 - val_loss: 0.4094 - val_acc: 0.8202
Epoch 33/40
1999872/2000000 [============================>.] - ETA: 0s - loss: 0.3091 - acc: 0.8613
Epoch 00033: val_acc did not improve from 0.82386
2000000/2000000 [==============================] - 43s 21us/sample - loss: 0.3091 - acc: 0.8613 - val_loss: 0.4164 - val_acc: 0.8200
Epoch 34/40
1998336/2000000 [============================>.] - ETA: 0s - loss: 0.3081 - acc: 0.8620
Epoch 00034: val_acc did not improve from 0.82386
2000000/20000

Epoch 15/40
1999744/2000000 [============================>.] - ETA: 0s - loss: 0.3220 - acc: 0.8541
Epoch 00015: val_acc did not improve from 0.82541
2000000/2000000 [==============================] - 40s 20us/sample - loss: 0.3220 - acc: 0.8541 - val_loss: 0.3870 - val_acc: 0.8232
Epoch 16/40
1997568/2000000 [============================>.] - ETA: 0s - loss: 0.3199 - acc: 0.8553
Epoch 00016: val_acc did not improve from 0.82541
2000000/2000000 [==============================] - 40s 20us/sample - loss: 0.3199 - acc: 0.8553 - val_loss: 0.3896 - val_acc: 0.8234
Epoch 17/40
1997056/2000000 [============================>.] - ETA: 0s - loss: 0.3179 - acc: 0.8565
Epoch 00017: val_acc did not improve from 0.82541
2000000/2000000 [==============================] - 40s 20us/sample - loss: 0.3179 - acc: 0.8565 - val_loss: 0.3945 - val_acc: 0.8233
Epoch 18/40
1998208/2000000 [============================>.] - ETA: 0s - loss: 0.3160 - acc: 0.8575
Epoch 00018: val_acc did not improve from 0.82541
2

In [33]:
filepath="models/convolutional_nn_layers_of_size_{}_{}".format(256, 30)
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

history = model.fit(X_train, y_train,
                    epochs=10,
                    verbose=True,
                    callbacks=callbacks_list,
                    validation_data=(X_test, y_test),
                    batch_size=128)

Train on 2000000 samples, validate on 500000 samples
Epoch 1/10
1999872/2000000 [============================>.] - ETA: 0s - loss: 0.4058 - acc: 0.8064
Epoch 00001: val_acc improved from -inf to 0.81485, saving model to models/convolutional_nn_layers_of_size_256_30
2000000/2000000 [==============================] - 1566s 783us/sample - loss: 0.4058 - acc: 0.8064 - val_loss: 0.3916 - val_acc: 0.8148
Epoch 2/10
1999872/2000000 [============================>.] - ETA: 0s - loss: 0.3780 - acc: 0.8225
Epoch 00002: val_acc improved from 0.81485 to 0.81871, saving model to models/convolutional_nn_layers_of_size_256_30
2000000/2000000 [==============================] - 1586s 793us/sample - loss: 0.3780 - acc: 0.8225 - val_loss: 0.3842 - val_acc: 0.8187
Epoch 3/10
1999872/2000000 [============================>.] - ETA: 0s - loss: 0.3597 - acc: 0.8328
Epoch 00003: val_acc improved from 0.81871 to 0.81954, saving model to models/convolutional_nn_layers_of_size_256_30
2000000/2000000 [=============

KeyboardInterrupt: 

In [ ]:
#########################
### don't add this

In [21]:
for i in range(30, 101, 10):

    model = Sequential()
    model.add(layers.Embedding(vocab_size, embedding_dim, 
                               weights=[embedding_matrix], 
                               input_length=maxlen, 
                               trainable=False))
    model.add(layers.GlobalMaxPool1D())
    model.add(layers.Dense(i, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    model.summary()



    filepath="models/random_nn_layers_of_size_{}_big".format(i)
    checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    callbacks_list = [checkpoint]

    history = model.fit(X_train, y_train,
                        epochs=30,
                        verbose=True,
                        validation_data=(X_test, y_test),
                        callbacks=callbacks_list,
                        batch_size=128)
    loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
    print("Training Accuracy: {:.4f}".format(accuracy))
    loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
    print("Testing Accuracy:  {:.4f}".format(accuracy))
    print("#############################\n############################\n##################################")


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 44, 200)           79397200  
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 200)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 30)                6030      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 31        
Total params: 79,403,261
Trainable params: 79,403,261
Non-trainable params: 0
_________________________________________________________________
Train on 2000000 samples, validate on 500000 samples
Epoch 1/30
1999872/2000000 [============================>.] - ETA: 0s - loss: 0.3982 - acc: 0.8114
Epoch 00001: val_acc improved from -inf to 0.82468, saving model to models/random_nn_layers_of_size_30_big
200

KeyboardInterrupt: 